In [ ]:
import requests # pip install requests
import json
import os
import time
from tqdm.notebook import tqdm # pip install tqdm

этот ноутбук спасет вашу слаку от забвения

In [ ]:
# вместо смайлика подставить user token из созданного слакового приложения. создать тут https://api.slack.com/apps/

In [ ]:
token = ':)'

In [ ]:
authorization=f"Bearer {token}"

# как получить последние сообщения с канала

In [ ]:
channel = 'C02J0TYKC21' # подставить свой
url = f"https://slack.com/api/conversations.history?channel={channel}" 
r = requests.post(url, headers={'Content-type': 'application/json', "Authorization": authorization})

In [ ]:
r.json()

# выкачать все личные сообщения

## получить маппинг пользователей на личные сообщения с ними

In [ ]:
url = "https://slack.com/api/conversations.list?types=im"
r = requests.post(url, headers={'Content-type': 'application/json', "Authorization": authorization})

In [ ]:
user_id_to_dm = {}
for ch in r.json()['channels']:
    user_id_to_dm[ch['user']] = ch['id']
print(user_id_to_dm)

In [ ]:
len(user_id_to_dm)

## получить маппинг пользователей на их имена

In [ ]:
url = "https://slack.com/api/users.list"
r = requests.post(url, headers={'Content-type': 'application/json',
                                "Authorization": authorization})

In [ ]:
user_id_to_name = {}
for member in r.json()['members']:
    user_id_to_name[member['id']] = member['profile']['real_name']
user_id_to_name

In [ ]:
len(user_id_to_name)

In [ ]:
result_dir = 'slack-history'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

## собсно качаем сообщения (пока без replies, но всё будет!)

In [ ]:
for i, (user_id, dm_id) in enumerate(tqdm(user_id_to_dm.items())):
    name = user_id_to_name.get(user_id, user_id)
    print(name)
    # именуем так чтобы каждый пользователь точно был уникальный
    # а то если человек уволился а потом снова устроился то будет две переписки с ним
    dir_name = os.path.join(result_dir, name + f"_{i}")
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    url = f"https://slack.com/api/conversations.history?channel={dm_id}&limit=200"
    r = requests.post(url, headers={'Content-type': 'application/json',
                                    "Authorization": authorization,
                                   })
    data = r.json()
    counter = 0
    number_of_messages = len(data['messages'])
    filename = os.path.join(dir_name, f"{counter}.json")

    with open(filename, 'w') as f:
        json.dump(data, f)
    response_metadata = data.get('response_metadata')
    if response_metadata is not None:
        next_cursor = data['response_metadata']['next_cursor']
        while next_cursor:
            counter += 1
            url = f"https://slack.com/api/conversations.history?channel={dm_id}&limit=200&cursor={next_cursor}"
            r = requests.post(url, headers={'Content-type': 'application/json',
                                            "Authorization": authorization,
                                           })
            data = r.json()
            filename = os.path.join(dir_name, f"{counter}.json")

            with open(filename, 'w') as f:
                json.dump(data, f)
            response_metadata = data.get('response_metadata')
            if response_metadata is not None:
                next_cursor = response_metadata['next_cursor'].replace("=", "%3D")
            else:
                next_cursor = ''
            number_of_messages += len(data['messages'])
            print(number_of_messages)
            time.sleep(0.3)